In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
import json

%matplotlib inline
pd.set_option('display.max_columns',500)
sns.set()

## Question 1: Does presence of smoke alarm improve survivability of fire
Null Hypothesis: There's no difference between survivability rates in fires with a smoke alarm and those without.

In [2]:
! dir ..\data\02_interim

 Volume in drive C is Acer
 Volume Serial Number is 14B0-52B3

 Directory of C:\Users\thwhi\Dropbox\Personal\Programming\DataKind Red Cross Project\data\02_interim

10/20/2019  12:42 AM    <DIR>          .
10/20/2019  12:42 AM    <DIR>          ..
10/19/2019  02:00 PM    <DIR>          .ipynb_checkpoints
10/20/2019  12:42 AM       250,251,601 01_nfirs_cleaned.csv
10/19/2019  03:50 PM             3,810 01_nfirs_data_dictionary.json
10/19/2019  08:31 PM       330,523,193 03_nfirs_geoids.csv
10/20/2019  12:42 AM    <DIR>          10000row_chunks
10/19/2019  07:03 PM    <DIR>          10000row_geoid_output
10/19/2019  01:55 PM            42,000 geo_batch_input.csv
10/19/2019  02:16 PM               460 geo_batch_input_10_rows.csv
10/20/2019  12:42 AM             3,810 nfirs_data_dictionary.json
10/19/2019  03:16 PM            36,452 test.ipynb
               7 File(s)    580,861,326 bytes
               5 Dir(s)   5,571,747,840 bytes free


In [3]:
nfirs = pd.read_csv('../data/02_interim/01_nfirs_cleaned.csv',dtype={})

with open('../data/02_interim/01_nfirs_data_dictionary.json','r') as f:
    nfirs_dictionary = json.load(f)

C:\Users\thwhi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (3,7,17,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
nfirs.head()

,Unnamed: 0,rcp2_id,state,fdid,inc_no,city,street,zip5,inc_date,oth_inj,oth_death,exp_no,inc_type,prop_loss,cont_loss,tot_loss,detector,det_type,det_power,det_operat,det_effect,det_fail,aes_pres,aes_type,aes_oper,no_spr_op,aes_fail
0,0,rcp2_id1,AK,11100,211,JUNEAU,1700 ANGUS WAY,99801,2009-01-03,0.0,0.0,0.0,111,20000.0,5000.0,25000.0,1,1,1,2,1,NaN,N,NaN,NaN,NaN,NaN
1,1,rcp2_id2,AK,11100,470,JUNEAU,3221 PIONEER AVE,99801,2009-01-06,0.0,0.0,0.0,114,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,rcp2_id3,AK,11100,556,JUNEAU,10231 HERON WAY,99801,2009-01-06,0.0,0.0,0.0,111,20000.0,5000.0,25000.0,N,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN
3,3,rcp2_id4,AK,11100,1315,JUNEAU,6590 GLACIER HWY,99801,2009-01-13,0.0,0.0,0.0,113,200.0,20.0,220.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,rcp2_id5,AK,11100,2151,JUNEAU,6590 GLACIER HWY,99801,2009-01-21,0.0,0.0,0.0,111,35000.0,5000.0,40000.0,N,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN


In [5]:
df = nfirs.iloc[:1000]

In [6]:
nfirs['detector'].value_counts(dropna=False)

NaN    925886
1      466610
U      301345
N      265265
Name: detector, dtype: int64

In [7]:
df = nfirs[(nfirs['detector'] == '1') | (nfirs['detector']=='N')].copy()

In [8]:
df['fatal'] = df['oth_death'] > 0

In [9]:
df_cross = pd.crosstab(df['detector'],df['fatal'])
df_cross

fatal,False,True
detector,,
1,463229,3381
N,263168,2097


In [10]:
df_cross['ratio'] = df_cross[True] / df_cross[False]

In [11]:
df_cross

fatal,False,True,ratio
detector,,,
1,463229,3381,0.007299
N,263168,2097,0.007968


In [14]:
1 - df_cross['ratio']['1'] / df_cross['ratio']['N']

0.08402403059414942

In [12]:
nfirs_dictionary

{'aes_fail': {'0': 'Reason system not effective, other',
  '1': 'System shut off',
  '2': 'Not enough agent discharged to control the fire',
  '3': 'Agent discharged, but did not reach the fire',
  '4': 'Inappropriate system for the type of fire',
  '5': 'Fire not in area protected by the system',
  '6': 'System components damaged',
  '7': 'Lack of maintenance, including corrosion or heads painted',
  '8': 'Manual intervention defeated the system',
  'U': 'Undetermined'},
 'aes_oper': {'0': 'Operation of AES, other',
  '1': 'System operated and was effective',
  '2': 'System operated and was not effective',
  '3': 'Fire too small to activate system',
  '4': 'System did not operate',
  'U': 'Undetermined'},
 'aes_pres': {'1': 'Present',
  '2': 'Partial system present',
  'N': 'None Present',
  'U': 'Undetermined'},
 'aes_type': {'0': 'Special hazard system, other',
  '1': 'Wet-pipe sprinkler system',
  '2': 'Dry-pipe sprinkler system',
  '3': 'Other sprinkler system',
  '4': 'Dry chemic